In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!ls

sample_data


In [96]:
%cd drive/My\ Drive/TryTextAttackAgain

[Errno 2] No such file or directory: 'drive/My Drive/TryTextAttackAgain'
/content/drive/My Drive/TryTextAttackAgain/TextAttack


In [97]:
!ls 

'=0.16.0'	    docs	 README.md	    textattack
 build		    examples	 requirements.txt   textattack.egg-info
 CONTRIBUTING.md    LICENSE	 setup.cfg	    wandb
 date_now-log.txt   Makefile	 setup.py
 dist		    pytest.ini	 tests


In [98]:
#!git clone https://github.com/QData/TextAttack.git

In [99]:
%cd TextAttack/

[Errno 2] No such file or directory: 'TextAttack/'
/content/drive/My Drive/TryTextAttackAgain/TextAttack


In [100]:
!pip install pyarrow>=0.16.0 --force-reinstall
!python3 setup.py install

ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: textattack 0.2.4 has requirement numpy<1.19.0, but you'll have numpy 1.19.1 which is incompatible.
ERROR: flair 0.6 has requirement pytest>=5.3.2, but you'll have pytest 3.6.4 which is incompatible.
running install
running bdist_egg
running egg_info
writing textattack.egg-info/PKG-INFO
writing dependency_links to textattack.egg-info/dependency_links.txt
writing entry points to textattack.egg-info/entry_points.txt
writing requirements to textattack.egg-info/requires.txt
writing top-level names to textattack.egg-info/top_level.txt
writing manifest file 'textattack.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
run

In [101]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.9.0          
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.0.1.post1    
astunparse                    1.6.3          
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         19.3.0         
audioread                     2.1.8          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bert-score                    0.3.5          
bleach                        3.1.5          
blis                          0.4.

# Create frequency dictionary


In [102]:
import textattack

In [103]:
import string
import nltk
import nlp
import csv
import os
import math 
from textattack.shared.utils import words_from_text
import numpy as np

def get_tokenize_sentences(premises,hypothesis):
  all_sentences = []

  for i in zip(premises,hypothesis):
    st = i[0] + " " + i[1]
    tokens = words_from_text(st)
    all_sentences.append(tokens)
  
  return all_sentences

def return_multi_nli_train_dict(dataset):

  train_data = nlp.load_dataset(dataset)['train']

  train_dict = {'premise':[],'hypothesis':[]}

  perc_ = math.ceil(len(train_data)*0.15)
  print("----EVAL DATASET USING : " + str(perc_) + "\n")

  eval_indices = np.sort(np.random.RandomState(21).choice(np.arange(len(train_data)),size=perc_,))
  train_indices = np.delete(np.arange(len(train_data)),eval_indices)

  train_dict['premise'] = np.array(train_data['premise'])[train_indices] 
  train_dict['hypothesis'] = np.array(train_data['hypothesis'])[train_indices]

  return train_dict

def create_freq_corpus(dataset,csvname):


  if dataset is 'multi_nli':
   print('MULTI') 
   train_data = return_multi_nli_train_dict(dataset)

  else:

    train_data = nlp.load_dataset(dataset)['train']

  
  all_sentences = get_tokenize_sentences(train_data['premise'],train_data['hypothesis'])

  freq_dict = {}

  for sentence in all_sentences:
    for token in sentence:
      if token  in freq_dict:
        freq_dict[token] += 1
      else:
        freq_dict[token] = 1 

  print("WRITE")
  with open(csvname, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in freq_dict.items():
      writer.writerow([key, value])
  

  return freq_dict

def load_csv(filename):

  with open(filename) as csv_file:
    reader = csv.reader(csv_file)
    freq_dict = dict(reader)
  
  return freq_dict



#SNLI

In [104]:
snli_file_name = '../snli_train_word_freq.csv'
snli_dataset_name = 'snli'
#snli_freq_dict = create_freq_corpus(snli_dataset_name,snli_file_name)
snli_freq_dict = load_csv(snli_file_name)

#MLNI

In [105]:
multi_file_name = '../multi_nli_train_word_freq.csv'
multi_dataset_name = 'multi_nli'
#multi_freq_dict = create_freq_corpus('multi_nli',multi_file_name)
multi_freq_dict = load_csv(multi_file_name)

In [106]:
snli_freq_dict['the']

'370390'

In [107]:
multi_freq_dict['the']

'515381'

#Get statistics from attack file, Table 2 shows the resulting log e frequencies Max's table

## We Added modifiable indices to attacked file

In [108]:
!ls ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack

alzantot_gpt2_100_attacked.csv
alzantot_gpt2_2000_attacked.csv
alzantot_gpt2_2000_attacked.gsheet
freq_attacks_snli_FrequencyP_2000_attacked.csv
freq_attacks_snli_FrequencyP_2_attacked.csv
freq_attacks_snli_FrequencyR_100_attacked.csv
freq_attacks_snli_FrequencyR_2000_attacked.csv
prioritized_Priotirized_100_attacked.csv
prioritized_Priotirized_2000_attacked.csv
prioritized_Priotirized_2_attacked.csv
pwws_snli_100_attacked.csv
pwws_snli_10_attacked.csv
pwws_snli_2000_attacked.csv
random_nli_100_attacked.csv
random_nli_10_attacked.csv
random_nli_1_attacked.csv
random_nli_2000_attacked.csv
random_nli_2000_attacked.gsheet
random_nli_200_attacked.csv
random_nli_303_attacked.csv
random_nli_50_attacked.csv
random_nlieval_whole_attacked.csv
textfooler_100_attacked.csv
textfooler_2000_attacked.csv


In [109]:
import pandas as pd
from ast import literal_eval
import numpy as np
import math 

def phi_value(freq):
  return math.log(1+freq)

def get_phi_values(words,freq_dict,in_dict=False):

  if in_dict:
    return [phi_value(int(freq_dict[word])) for word in words if word in freq_dict]

  return [phi_value(int(freq_dict[word])) if word in freq_dict else 0 for word in words]


def get_attack_freq_stats(filename,freq_dict):
  
  attack_data = pd.read_csv(filename)

  premises = [i['premise'] for _, i in attack_data.iterrows()] 
  attacked_hypo = [i['attacked_hypothesis'] for _, i in attack_data.iterrows()] 
  original_hypo = [i['original_hypothesis'] for _, i in attack_data.iterrows()] 
  modifiable_indices = [literal_eval(i['modifiable_indices']) for _, i in attack_data.iterrows()] 
  modified_indices = [literal_eval(i['modified_indices']) for _, i in attack_data.iterrows()] 

  attacked_sentences = get_tokenize_sentences(premises,attacked_hypo)
  original_sentences = get_tokenize_sentences(premises,original_hypo)
  
  replaceable_words =[]
  replaced_words = []
  sub_words = []

  for i in range(len(original_sentences)):
    replaceable_words.append(np.array(original_sentences[i])[modifiable_indices[i]])
    sub_words.append(np.array(attacked_sentences[i])[modified_indices[i]])
    replaced_words.append(np.array(original_sentences[i])[modified_indices[i]])

  replaceable_words_freqs = []
  replaced_words_freqs = []
  sub_words_freqs = []
  sub_ovv_freqs = []

  for i in range(len(replaceable_words)):
    replaceable_words_freqs = np.concatenate((replaceable_words_freqs,get_phi_values(replaceable_words[i],freq_dict)))
    replaced_words_freqs = np.concatenate((replaced_words_freqs,get_phi_values(replaced_words[i],freq_dict)))
    sub_words_freqs = np.concatenate((sub_words_freqs,get_phi_values(sub_words[i],freq_dict)))
    sub_ovv_freqs = np.concatenate((sub_ovv_freqs,get_phi_values(sub_words[i],freq_dict, True)))

  print("MEAN:\t" + str(np.mean(replaceable_words_freqs)) + "\t STD:\t" + str(np.std(replaceable_words_freqs)))
  print("MEAN:\t" + str(np.mean(replaced_words_freqs)) + "\t STD:\t" + str(np.std(replaced_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_words_freqs)) + "\t STD:\t" + str(np.std(sub_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_ovv_freqs)) + "\t STD:\t" + str(np.std(sub_ovv_freqs)))

  #return replaceable_words_freqs,replaced_words_freqs,sub_words_freqs,sub_ovv_freqs
  


In [110]:
!ls ../TrainedBertModels/outputs/training/attack/

ls: cannot access '../TrainedBertModels/outputs/training/attack/': No such file or directory


In [111]:
!ls ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/

alzantot_gpt2_100_attacked.csv
alzantot_gpt2_2000_attacked.csv
alzantot_gpt2_2000_attacked.gsheet
freq_attacks_snli_FrequencyP_2000_attacked.csv
freq_attacks_snli_FrequencyP_2_attacked.csv
freq_attacks_snli_FrequencyR_100_attacked.csv
freq_attacks_snli_FrequencyR_2000_attacked.csv
prioritized_Priotirized_100_attacked.csv
prioritized_Priotirized_2000_attacked.csv
prioritized_Priotirized_2_attacked.csv
pwws_snli_100_attacked.csv
pwws_snli_10_attacked.csv
pwws_snli_2000_attacked.csv
random_nli_100_attacked.csv
random_nli_10_attacked.csv
random_nli_1_attacked.csv
random_nli_2000_attacked.csv
random_nli_2000_attacked.gsheet
random_nli_200_attacked.csv
random_nli_303_attacked.csv
random_nli_50_attacked.csv
random_nlieval_whole_attacked.csv
textfooler_100_attacked.csv
textfooler_2000_attacked.csv


In [112]:
#!textattack eval --num-examples 50 --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --from_snli_attak_file ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/random_nli_10_attacked.csv

In [113]:
#!textattack eval --from_file_attack_or_original original --num-examples 50 --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --from_snli_attak_file ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/random_nli_10_attacked.csv

# Frequency and other Attacks (random,prioritizied, azlanotgpt, textfooler, pwwssnli)

In [114]:
!ls ../

 BaseLoading.ipynb
'Copy of DefendBertGuide.ipynb'
 corpus.py
 DefendBertGuide.ipynb
 detect_adversarial.py
 EvalBert.ipynb
 freq_attacks_mnli.py
 freq_attacks_snli.py
 freq_stats.py
 InferSent.ipynb
 inferSentModel.py
 InferSent_snli_pretrained_GLOVEepoch10.fullmodel
 multi_nli_train_word_freq.csv
 preprocess_datasets
 prioritized.py
 __pycache__
 snli_train_word_freq.csv
 TextAttack
 TrainedBertModels
 TrainNotebooks
 wandb


In [115]:
!ls ../

 BaseLoading.ipynb
'Copy of DefendBertGuide.ipynb'
 corpus.py
 DefendBertGuide.ipynb
 detect_adversarial.py
 EvalBert.ipynb
 freq_attacks_mnli.py
 freq_attacks_snli.py
 freq_stats.py
 InferSent.ipynb
 inferSentModel.py
 InferSent_snli_pretrained_GLOVEepoch10.fullmodel
 multi_nli_train_word_freq.csv
 preprocess_datasets
 prioritized.py
 __pycache__
 snli_train_word_freq.csv
 TextAttack
 TrainedBertModels
 TrainNotebooks
 wandb


In [116]:
%cd ..
!ls 
%cd TextAttack
!ls

/content/drive/My Drive/TryTextAttackAgain
 BaseLoading.ipynb
'Copy of DefendBertGuide.ipynb'
 corpus.py
 DefendBertGuide.ipynb
 detect_adversarial.py
 EvalBert.ipynb
 freq_attacks_mnli.py
 freq_attacks_snli.py
 freq_stats.py
 InferSent.ipynb
 inferSentModel.py
 InferSent_snli_pretrained_GLOVEepoch10.fullmodel
 multi_nli_train_word_freq.csv
 preprocess_datasets
 prioritized.py
 __pycache__
 snli_train_word_freq.csv
 TextAttack
 TrainedBertModels
 TrainNotebooks
 wandb
/content/drive/My Drive/TryTextAttackAgain/TextAttack
'=0.16.0'	    docs	 README.md	    textattack
 build		    examples	 requirements.txt   textattack.egg-info
 CONTRIBUTING.md    LICENSE	 setup.cfg	    wandb
 date_now-log.txt   Makefile	 setup.py
 dist		    pytest.ini	 tests


In [117]:
!ls ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack

alzantot_gpt2_100_attacked.csv
alzantot_gpt2_2000_attacked.csv
alzantot_gpt2_2000_attacked.gsheet
freq_attacks_snli_FrequencyP_2000_attacked.csv
freq_attacks_snli_FrequencyP_2_attacked.csv
freq_attacks_snli_FrequencyR_100_attacked.csv
freq_attacks_snli_FrequencyR_2000_attacked.csv
prioritized_Priotirized_100_attacked.csv
prioritized_Priotirized_2000_attacked.csv
prioritized_Priotirized_2_attacked.csv
pwws_snli_100_attacked.csv
pwws_snli_10_attacked.csv
pwws_snli_2000_attacked.csv
random_nli_100_attacked.csv
random_nli_10_attacked.csv
random_nli_1_attacked.csv
random_nli_2000_attacked.csv
random_nli_2000_attacked.gsheet
random_nli_200_attacked.csv
random_nli_303_attacked.csv
random_nli_50_attacked.csv
random_nlieval_whole_attacked.csv
textfooler_100_attacked.csv
textfooler_2000_attacked.csv


In [118]:
#!textattack attack --attack_output_snli ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --recipe random_nli  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing

In [119]:
get_attack_freq_stats('../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/random_nli_2000_attacked.csv',snli_freq_dict)

MEAN:	8.143241883166127	 STD:	2.393379678926035
MEAN:	8.300980741810948	 STD:	2.28266261082952
MEAN:	3.2564202445139436	 STD:	3.158630428259514
MEAN:	4.846917544201015	 STD:	2.6722399444525218


In [120]:
#!textattack attack --attack_output_snli ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --attack-from-class ../prioritized Priotirized  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing

In [121]:
get_attack_freq_stats('../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/prioritized_Priotirized_2000_attacked.csv',snli_freq_dict)

MEAN:	8.143241883166127	 STD:	2.393379678926035
MEAN:	8.400799758802355	 STD:	2.231744520302462
MEAN:	2.81911605516509	 STD:	2.8971680669332494
MEAN:	4.3381835802576125	 STD:	2.5152399335876514


In [122]:
#!textattack attack --attack_output_snli ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --attack-from-class ../freq_attacks_snli FrequencyR  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing

In [123]:
get_attack_freq_stats('../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/freq_attacks_snli_FrequencyR_2000_attacked.csv',snli_freq_dict)

MEAN:	8.143241883166127	 STD:	2.393379678926035
MEAN:	8.2138734498559	 STD:	2.3058534276290015
MEAN:	0.175582441304962	 STD:	0.8123246416428814
MEAN:	2.221614813945425	 STD:	1.9503189412876354


In [124]:
#!textattack attack --attack_output_snli ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --attack-from-class ../freq_attacks_snli FrequencyP  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing

In [125]:
get_attack_freq_stats('../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/freq_attacks_snli_FrequencyP_2000_attacked.csv',snli_freq_dict)

MEAN:	8.143241883166127	 STD:	2.393379678926035
MEAN:	8.165597752614305	 STD:	2.2948832411195452
MEAN:	0.1795608744299696	 STD:	0.8111355229337271
MEAN:	2.1001439873329244	 STD:	1.9135764807625557


In [126]:
#!textattack attack --attack_output_snli ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --recipe textfooler  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing

In [127]:
get_attack_freq_stats('../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/textfooler_2000_attacked.csv',snli_freq_dict)

MEAN:	9.049272434260248	 STD:	2.7331396093253293
MEAN:	8.253414516935555	 STD:	2.4467484207714887
MEAN:	2.5682893946746606	 STD:	2.8431054493520405
MEAN:	4.093955346345634	 STD:	2.576627868231009


In [128]:
#!textattack attack --attack_output_snli ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --recipe pwws_snli  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing

In [129]:
get_attack_freq_stats('../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/pwws_snli_2000_attacked.csv',snli_freq_dict)

MEAN:	9.049272434260248	 STD:	2.7331396093253293
MEAN:	8.894910637501537	 STD:	2.438250711462463
MEAN:	2.6265573441856755	 STD:	2.9233686893469755
MEAN:	4.336654032306565	 STD:	2.587308275027282


In [130]:
#!textattack attack --attack_output_snli ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --recipe alzantot_gpt2  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing

In [131]:
get_attack_freq_stats('../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/alzantot_gpt2_2000_attacked.csv',snli_freq_dict)

MEAN:	9.049272434260248	 STD:	2.7331396093253293
MEAN:	8.688605384728167	 STD:	2.396477294737005
MEAN:	3.230326621751251	 STD:	3.3206730862379263
MEAN:	4.740259595706191	 STD:	3.003934372482202


# Try implementing FWGS

In [132]:
snli_freq_dict['the']

'370390'

In [133]:
multi_freq_dict['the']

'515381'

In [134]:
import pandas as pd
from ast import literal_eval
import numpy as np
import math 

def phi_value(freq):
  return math.log(1+freq)

def get_phi_values(words,freq_dict,in_dict=False):

  if in_dict:
    return [phi_value(int(freq_dict[word])) for word in words if word in freq_dict]

  return [phi_value(int(freq_dict[word])) if word in freq_dict else 0 for word in words]


def get_attack_freq_stats(filename,freq_dict):
  
  attack_data = pd.read_csv(filename)

  premises = [i['premise'] for _, i in attack_data.iterrows()] 
  attacked_hypo = [i['attacked_hypothesis'] for _, i in attack_data.iterrows()] 
  original_hypo = [i['original_hypothesis'] for _, i in attack_data.iterrows()] 
  modifiable_indices = [literal_eval(i['modifiable_indices']) for _, i in attack_data.iterrows()] 
  modified_indices = [literal_eval(i['modified_indices']) for _, i in attack_data.iterrows()] 

  attacked_sentences = get_tokenize_sentences(premises,attacked_hypo)
  original_sentences = get_tokenize_sentences(premises,original_hypo)

  replaceable_words =[]
  replaced_words = []
  sub_words = []

  for i in range(len(original_sentences)):
    replaceable_words.append(np.array(original_sentences[i])[modifiable_indices[i]])
    sub_words.append(np.array(attacked_sentences[i])[modified_indices[i]])
    replaced_words.append(np.array(original_sentences[i])[modified_indices[i]])

  replaceable_words_freqs = []
  replaced_words_freqs = []
  sub_words_freqs = []
  sub_ovv_freqs = []

  for i in range(len(replaceable_words)):
    replaceable_words_freqs = np.concatenate((replaceable_words_freqs,get_phi_values(replaceable_words[i],freq_dict)))
    replaced_words_freqs = np.concatenate((replaced_words_freqs,get_phi_values(replaced_words[i],freq_dict)))
    sub_words_freqs = np.concatenate((sub_words_freqs,get_phi_values(sub_words[i],freq_dict)))
    sub_ovv_freqs = np.concatenate((sub_ovv_freqs,get_phi_values(sub_words[i],freq_dict, True)))

  print("MEAN:\t" + str(np.mean(replaceable_words_freqs)) + "\t STD:\t" + str(np.std(replaceable_words_freqs)))
  print("MEAN:\t" + str(np.mean(replaced_words_freqs)) + "\t STD:\t" + str(np.std(replaced_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_words_freqs)) + "\t STD:\t" + str(np.std(sub_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_ovv_freqs)) + "\t STD:\t" + str(np.std(sub_ovv_freqs)))

  #return replaceable_words_freqs,replaced_words_freqs,sub_words_freqs,sub_ovv_freqs
  


In [135]:
#azalont_attack_ = pd.read_csv('../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/alzantot_gpt2_2000_attacked.csv')

In [136]:
#azalont_attack_

In [137]:
#!textattack eval --attacked_or_original attacked --num-examples 50 --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --attack-from-class ../prioritized Priotirized --from_snli_attak_file ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/random_nli_10_attacked.csv 

In [138]:
#!textattack eval --attacked_or_original original --num-examples 50 --model-from-huggingface ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing --attack-from-class ../prioritized Priotirized --from_snli_attak_file ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/random_nli_10_attacked.csv 

In [139]:
import argparse
import copy
import importlib
import json
import os
import time
import math 

import textattack

from textattack.commands.attack.attack_args import (
    ATTACK_RECIPE_NAMES,
    BLACK_BOX_TRANSFORMATION_CLASS_NAMES,
    CONSTRAINT_CLASS_NAMES,
    GOAL_FUNCTION_CLASS_NAMES,
    HUGGINGFACE_DATASET_BY_MODEL,
    SEARCH_METHOD_CLASS_NAMES,
    TEXTATTACK_DATASET_BY_MODEL,
    WHITE_BOX_TRANSFORMATION_CLASS_NAMES,
)

# The split token allows users to optionally pass multiple arguments in a single
# parameter by separating them with the split token.
ARGS_SPLIT_TOKEN = "^"

def _cb(s):
  return textattack.shared.utils.color_text(str(s), color="blue", method="ansi")

def phi_value(freq):
    return math.log(1+freq)

def load_module_from_file(file_path):
    """Uses ``importlib`` to dynamically open a file and load an object from
    it."""
    temp_module_name = f"temp_{time.time()}"
    colored_file_path = textattack.shared.utils.color_text(
        file_path, color="blue", method="ansi"
    )
    textattack.shared.logger.info(f"Loading module from `{colored_file_path}`.")
    spec = importlib.util.spec_from_file_location(temp_module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def parse_model(model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

    if model_from_file:
        # Support loading the model from a .py file where a model wrapper
        # is instantiated.
        colored_model_name = textattack.shared.utils.color_text(
            model_from_file, color="blue", method="ansi"
        )
        textattack.shared.logger.info(
            f"Loading model and tokenizer from file: {colored_model_name}"
        )
        if ARGS_SPLIT_TOKEN in model_from_file:
            model_file, model_name = model_from_file.split(ARGS_SPLIT_TOKEN)
        else:
            _, model_name = model_from_file, "model"
        try:
            model_module = load_module_from_file(model_from_file)
            print("))))HERE(((())))")
            print(model_module)
        except Exception:
            raise ValueError(f"Failed to import file {model_from_file}")
        try:
            model = getattr(model_module, model_name)
        except AttributeError:
            raise AttributeError(
                f"``{model_name}`` not found in module {model_from_file}"
            )

        if not isinstance(model, textattack.models.wrappers.ModelWrapper):
            
            if isinstance(model, textattack.models.NLINet):
                
                model = textattack.models.wrappers.PyTorchModelWrapper(
                    model, model.encoder, batch_size=model_batch_size
                )
                
            else:
                raise TypeError(
                    "Model must be of type "
                    f"``textattack.models.ModelWrapper``, got type {type(model)}"
                )

          
    elif (model in HUGGINGFACE_DATASET_BY_MODEL) or model_from_huggingface:
        # Support loading models automatically from the HuggingFace model hub.
        import transformers

        model_name = (
            HUGGINGFACE_DATASET_BY_MODEL[args.model][0]
            if (model in HUGGINGFACE_DATASET_BY_MODEL)
            else model_from_huggingface
        )

        if ARGS_SPLIT_TOKEN in model_name:
            model_class, model_name = model_name
            model_class = eval(f"transformers.{model_class}")
        else:
            model_class, model_name = (
                transformers.AutoModelForSequenceClassification,
                model_name,
            )
        colored_model_name = textattack.shared.utils.color_text(
            model_name, color="blue", method="ansi"
        )
        textattack.shared.logger.info(
            f"Loading pre-trained model from HuggingFace model repository: {colored_model_name}"
        )
        model = model_class.from_pretrained(model_name)
        tokenizer = textattack.models.tokenizers.AutoTokenizer(model_name)
        model = textattack.models.wrappers.HuggingFaceModelWrapper(
            model, tokenizer, batch_size=model_batch_size
        )
    elif model in TEXTATTACK_DATASET_BY_MODEL:
        # Support loading TextAttack pre-trained models via just a keyword.
        model_path, _ = TEXTATTACK_DATASET_BY_MODEL[model]
        model = textattack.shared.utils.load_textattack_model_from_path(
            model, model_path
        )
        # Choose the approprate model wrapper (based on whether or not this is
        # a HuggingFace model).
        if isinstance(
            model, textattack.models.helpers.BERTForClassification
        ) or isinstance(model, textattack.models.helpers.T5ForTextToText):
            model = textattack.models.wrappers.HuggingFaceModelWrapper(
                model, model.tokenizer, batch_size=model_batch_size
            )
        else:
            model = textattack.models.wrappers.PyTorchModelWrapper(
                model, model.tokenizer, batch_size=model_batch_size
            )
    else:
      raise ValueError("Nothing else supported")
    

    return model

def parse_defense_from_class(defense_from_class=None,freq_dict=None,delta=10,type='discrete',gamma='0.1',candidates=8,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

    model = parse_model(model_from_file, model,model_from_huggingface,model_batch_size)
    
    if defense_from_class:
      import sys
      path = defense_from_class[0]
      path_class = path.rsplit('/')
      folder_path =  path_class[0] + '/'
      file = path_class[1]

      sys.path.insert(0,folder_path)
      module = importlib.import_module(file)
      my_class = getattr(module, defense_from_class[1])

      return my_class.build(model,freq_dict,delta=delta,type=type,gamma=gamma,candidates=candidates)
    else:
      raise ValueError("currently only attack from class is supported")

In [140]:
import sys
sys.path.insert(0,"../")

In [152]:
import scipy
import torch
import pandas as pd
from collections import deque
import corpus
import tqdm
import time
import textattack

logger = textattack.shared.logger

column_encoder = {"Entailment":0, "Neutral":1, "Contradiction":2}
reversed_encoder = {0:"Entailment", 1:"Neutral", 2:"Contradiction"}

def parse_log_manager():
    # Create logger
    attack_log_manager = textattack.loggers.AttackLogManager()

    '''
    # Get current time for file naming
    timestamp = time.strftime("%Y-%m-%d-%H-%M")

    # Get default directory to save results
    current_dir = os.path.dirname(os.path.realpath(__file__))
    outputs_dir = os.path.join(
        current_dir, os.pardir, os.pardir, os.pardir, "outputs", "attacks"
    )
    out_dir_txt = out_dir_csv = os.path.normpath(outputs_dir)

    # Get default txt and csv file names
    filename_txt = f"_{timestamp}.txt"
    filename_csv = f"_{timestamp}.csv"
    
    

    # in case directory doesn't exist
    if not os.path.exists(out_dir_txt):
        os.makedirs(out_dir_txt)
    if not os.path.exists(out_dir_csv):
        os.makedirs(out_dir_csv)

    attack_log_manager.add_output_file(os.path.join(out_dir_txt, filename_txt))
    '''
    #attack_log_manager.enable_visdom()
    #attack_log_manager.enable_wandb()
    attack_log_manager.enable_stdout()
    return attack_log_manager

def evaluate_set(attacked_dataset,attack):

  num_remaining_attacks = len(attacked_dataset)
  worklist = deque(range(0, len(attacked_dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0

  load_time = time.time()


  for result in attack.attack_dataset(attacked_dataset, indices=worklist):

    initial_score = 1-result.original_result.score
    print(initial_score)
    print(result.original_result.attacked_text)
    pertubated_score = 1-result.perturbed_result.score
    print(pertubated_score)
    print(result.perturbed_result.attacked_text)
    attack_log_manager.log_result(result)

    print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

  pbar.close()

  # Enable summary stdout
  
  attack_log_manager.log_summary()
  attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  return num_successes, num_failures



class EvalDefenseModel():

  def __init__(
      self,
      defense_from_class,
      model_from_file=None, 
      model=None,
      model_from_huggingface=None,
      model_batch_size=32,
      delta = 10,
      gamma = 1,
      type = 'discrete',
      freq_dict = None,
      data_file = None,
      data_name = "snli",
      candidates = 8,
      #indices_to_modify=None,
      #shifted_idxs=True,
    ):

    # Get attack
    attack = parse_defense_from_class(defense_from_class,freq_dict,delta,type,gamma,candidates,model_from_file, model,model_from_huggingface,model_batch_size)
    
    #Get datasets
    attacked_data = pd.read_csv(data_file,usecols=["premise",'attacked_hypothesis', 'modified_indices' , "attacked_label"])
    to_remove = []
    for index,x in attacked_data.iterrows():
      if x['modified_indices'] == '[]':
        to_remove.append(index)
    attacked_data = attacked_data.drop(to_remove)
    attacked_data = attacked_data.drop(['modified_indices'],axis=1)

    attacked_data["attacked_label"] = [column_encoder[i] for i in attacked_data["attacked_label"] ]


    original_data = pd.read_csv(data_file,usecols=["premise",'original_hypothesis', "original_label"])
    original_data["original_label"] = [column_encoder[i] for i in original_data["original_label"] ]

    attacked_data = attacked_data.rename(columns={'attacked_hypothesis': 'hypothesis','attacked_label':'label'})
    attacked_dataset = nlp.Dataset.from_pandas(attacked_data)

    attacked_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=attacked_dataset, label_map = reversed_encoder)
    
    original_data = original_data.rename(columns={'original_hypothesis': 'hypothesis','original_label':'label'})
    original_dataset = nlp.Dataset.from_pandas(original_data)

    original_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=original_dataset, label_map = reversed_encoder)
    
    print("------Attacked Dataset------")
    true_positives, false_negatives = evaluate_set(attacked_dataset,attack)
    print("------Original Dataset------")
    false_positives, true_negatives = evaluate_set(original_dataset,attack)
    

    print("True positives :" + str(true_positives) + "\n")
    print("False_negatives :" + str(false_negatives) + "\n")
    print("False positives :" + str(false_positives) + "\n")
    print("True negative :" + str(true_negatives) + "\n")

    self.tp = true_positives
    self.fn = false_negatives
    self.fp = false_positives
    self.tn = true_negatives
    

In [153]:
list(column_encoder.keys())

['Entailment', 'Neutral', 'Contradiction']

In [154]:
a = nlp.load_dataset('snli',split='train').features["label"]
a.names

['entailment', 'neutral', 'contradiction']

In [155]:
print('Try loading SNLI training Corpus')
snli_freq_dict = corpus.load_csv('../snli_train_word_freq.csv')

stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
      model_from_file=None, 
      model=None,
      model_from_huggingface='../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing',
      model_batch_size=32,
      delta = phi_value(50),
      type = 'discrete',
      gamma = 1,
      freq_dict = snli_freq_dict,
      data_file='../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/alzantot_gpt2_2000_attacked.csv',
      data_name = 'snli',
      candidates =8)

Output hidden; open in https://colab.research.google.com to view.

In [93]:
print(stat.tp)
print(stat.fp)
print(stat.fn)
print(stat.tn)
fpr = stat.fp / (stat.fp + stat.tn)
tpr = stat.tp / (stat.tp + stat.fn)
print(fpr)
print(tpr)


859
38
905
1962
0.019
0.48696145124716556


In [ ]:
!ls ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing

1597174600.3340306				    README.md
attack						    special_tokens_map.json
config.json					    tokenizer_config.json
events.out.tfevents.1597174600.711b874ec62f.3718.0  train_args.json
log.txt						    vocab.txt
pytorch_model.bin


In [ ]:
print('Try loading SNLI training Corpus')
snli_freq_dict = corpus.load_csv('../snli_train_word_freq.csv')

EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
      model_from_file=None, 
      model=None,
      model_from_huggingface='../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing',
      model_batch_size=32,
      delta = phi_value(50),
      type = 'continuous',
      gamma = 0.1,
      freq_dict = snli_freq_dict,
      data_file='../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/alzantot_gpt2_2000_attacked.csv',
      data_name = 'snli',
      candidates=8)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing


Try loading SNLI training Corpus


textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading nlp dataset snli, split None.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 4 / 2 / 6:   0%|          | 6/1764 [00:00<00:56, 31.21it/s]

...

--------------------------------------------- Result 1986 ---------------------------------------------
Contradiction (100%) --> [FAILED]

Premise: A male windsurfer surfing on some blue waves with his yellow surfboard and red and white sail it looks as though he is about to fall into the water because he is suspended in the air sideways.
Hypothesis: A woman swims by herself through the waves.


--------------------------------------------- Result 1987 ---------------------------------------------
Contradiction (100%) --> [FAILED]

Premise: A cart of large water containers is in front of two women who are standing as well as two women who are walking down the street.
Hypothesis: The cart is full of beer.


--------------------------------------------- Result 1988 ---------------------------------------------
Contradiction (100%) --> [FAILED]

Premise: In front is a guy singing and playing guitar and a white shirted drummer in the back.
Hypothesis: The people are sleeping.


-----------

[Succeeded / Failed / Total] 58 / 1942 / 2000: 100%|██████████| 2000/2000 [00:33<00:00, 60.58it/s]

--------------------------------------------- Result 1998 ---------------------------------------------
Neutral (87%) --> [FAILED]

Premise: A woman is cutting into a sandwich while sitting at an outdoor table.
Hypothesis: the lady on the table is leeping


--------------------------------------------- Result 1999 ---------------------------------------------
Neutral (81%) --> Contradiction (87%)

Premise: Politician John McCain on a talk show.
Hypothesis: John McCain is talking about politics.

Premise: Politician John McCain on a talk show.
Hypothesis: John obama is talking about politics.


--------------------------------------------- Result 2000 ---------------------------------------------
Entailment (99%) --> [FAILED]

Premise: A boy in a green shirt is sitting in an almost empty stadium.
Hypothesis: A boy is sitting down.


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful atta


textattack: Attack time: 33.0216383934021s



True positives :920

False_negatives :844

False positives :58

True negative :1942



In [ ]:
!ls ../

 BaseLoading.ipynb
'Copy of DefendBertGuide.ipynb'
 corpus.py
 DefendBertGuide.ipynb
 detect_adversarial.py
 EvalBert.ipynb
 eval_datasets
 freq_attacks_mnli.py
 freq_attacks_snli.py
 freq_stats.py
 InferSent.ipynb
 inferSentModel.py
 InferSent_snli_pretrained_GLOVEepoch10.fullmodel
 multi_nli_train_word_freq.csv
 preprocess_datasets
 prioritized.py
 __pycache__
 snli_train_word_freq.csv
 TextAttack
 TrainBert.ipynb
 TrainedBertModels
 TrainedCNN
 TrainedLSTM
 TryBertMultiNLI.ipynb
 wandb


In [ ]:
%cd ..
!ls 
%cd TextAttack
!ls

/content/drive/My Drive/TryTextAttackAgain
 BaseLoading.ipynb
'Copy of DefendBertGuide.ipynb'
 corpus.py
 DefendBertGuide.ipynb
 detect_adversarial.py
 EvalBert.ipynb
 eval_datasets
 freq_attacks_mnli.py
 freq_attacks_snli.py
 freq_stats.py
 InferSent.ipynb
 inferSentModel.py
 InferSent_snli_pretrained_GLOVEepoch10.fullmodel
 multi_nli_train_word_freq.csv
 preprocess_datasets
 prioritized.py
 __pycache__
 snli_train_word_freq.csv
 TextAttack
 TrainBert.ipynb
 TrainedBertModels
 TrainedCNN
 TrainedLSTM
 TryBertMultiNLI.ipynb
 wandb
/content/drive/My Drive/TryTextAttackAgain/TextAttack
'=0.16.0'	    docs	 README.md	    textattack
 build		    examples	 requirements.txt   textattack.egg-info
 CONTRIBUTING.md    LICENSE	 setup.cfg	    wandb
 date_now-log.txt   Makefile	 setup.py
 dist		    pytest.ini	 tests


In [ ]:
'''
EvalDefenseModel(defense_from_class=["../prioritized", "Priotirized"],
     model_from_file="../inferSentModel.py", , 
      model=None,
      model_from_huggingface = None,
      model_batch_size=32,
      delta = 10,
      gamma = 1,
      freq_dict = None)
'''

'\nEvalDefenseModel(defense_from_class=["../prioritized", "Priotirized"],\n     model_from_file="../inferSentModel.py", , \n      model=None,\n      model_from_huggingface = None,\n      model_batch_size=32,\n      delta = 10,\n      gamma = 1,\n      freq_dict = None)\n'

# Hyperparameter tuning

## Create Valid datasets used for tuning

In [ ]:
def return_multi_nli_eval_dict(dataset):

  train_data = nlp.load_dataset(dataset)['train']

  valid_dict = {'premise':[],'hypothesis':[]}

  perc_ = math.ceil(len(train_data)*0.15)
  print("----EVAL DATASET USING : " + str(perc_) + "\n")

  eval_indices = np.sort(np.random.RandomState(21).choice(np.arange(len(train_data)),size=perc_,))

  valid_dict['premise'] = np.array(train_data['premise'])[eval_indices] 
  valid_dict['hypothesis'] = np.array(train_data['hypothesis'])[eval_indices]
  valid_dict['label'] = np.array(train_data['label'])[eval_indices]


  return valid_dict

In [ ]:
# Create Eval datasets
snli_dataset = nlp.load_dataset("snli")

multi_eval_dataset = return_multi_nli_eval_dict("multi_nli")
snli_eval_dataset = snli_dataset['validation']
snli_eval_dataset = {'premise':snli_eval_dataset['premise'],'hypothesis':snli_eval_dataset['hypothesis'],'label':snli_eval_dataset['label']}

----EVAL DATASET USING : 58906



In [68]:
from textattack.shared.utils import get_modifiable_indices

def parse_attack_from_recipe(recipe=None,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

  model = parse_model(model_from_file, model,model_from_huggingface,model_batch_size)

  recipe = eval(f"{ATTACK_RECIPE_NAMES[recipe]}.build(model)")
  recipe.goal_function.query_budget = float("inf")
  recipe.goal_function.model_batch_size = model_batch_size
  recipe.goal_function.model_cache_size = 2 ** 18
  recipe.constraint_cache_size = 2 ** 18
  return recipe
    
def create_snli_dict():
    dictionary = {}
    dictionary["premise"] = []
    dictionary["original_hypothesis"] = []
    dictionary["attacked_hypothesis"] = []
    dictionary["label"] = []
    dictionary["original_label"] = []
    dictionary["attacked_label"] = []
    dictionary['modifiable_indices'] = []
    dictionary['modified_indices'] = []

    return dictionary

def create_csv_file(dictionary,filename):
    df = pd.DataFrame(dictionary)
    df.to_csv(filename, encoding='utf-8', index=False)
    return df

def attack_recipe(recipe,dataset,data_name,folder,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):
     
  # Get attack
  attack = parse_attack_from_recipe(recipe,model_from_file, model,model_from_huggingface,model_batch_size)
    
  #Remove untified instances
  to_remove = []
  for index,x in dataset.iterrows():
    if x['label'] == -1:
      to_remove.append(index)

  dataset = dataset.drop(to_remove)
  dataset = nlp.Dataset.from_pandas(dataset)

  dataset = textattack.datasets.HuggingFaceNlpDataset(
          name=data_name, shuffle=False, split=None, dataset=dataset, label_map = reversed_encoder)
    
  num_remaining_attacks = len(dataset)
  worklist = deque(range(0, len(dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0

  load_time = time.time()

  output_attack_dir = os.path.join(folder, "attack")
  output_attack_dir = os.path.normpath(output_attack_dir)
  if not os.path.exists(output_attack_dir):
    os.makedirs(output_attack_dir)

  print("\nAttacks will be saved at " + output_attack_dir + "\n")
  attacked_dict = create_snli_dict()

  for result in attack.attack_dataset(dataset, indices=worklist):

    modifiable_indices = get_modifiable_indices(result.original_result.attacked_text,attack.pre_transformation_constraints)
    prem = result.original_result.attacked_text._text_input['premise']
    hypothesis = result.original_result.attacked_text._text_input['hypothesis']
    attacked_dict['premise'].append(prem)
    attacked_dict['original_hypothesis'].append(hypothesis)
    result.change_word_numbers = len(hypothesis.split())
    attacked_dict['label'].append(result.original_result.ground_truth_output)
    attacked_dict['original_label'].append(result.original_result._processed_output[0])
    attacked_dict['modifiable_indices'].append(list(modifiable_indices))

    if isinstance(result, textattack.attack_results.SkippedAttackResult):
        attacked_dict['attacked_hypothesis'].append(hypothesis)
        attacked_dict['attacked_label'].append(result.original_result._processed_output[0])
        attacked_dict['modified_indices'].append([])

    else:

        hypothesis = result.perturbed_result.attacked_text._text_input['hypothesis']
        attacked_dict['attacked_hypothesis'].append(hypothesis)
        attacked_dict['attacked_label'].append(result.perturbed_result._processed_output[0])
        attacked_dict['modified_indices'].append(list(result.perturbed_result.attacked_text.attack_attrs['modified_indices']))

   
    attack_log_manager.log_result(result)

    print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

  pbar.close()

  # Enable summary stdout
  
  attack_log_manager.log_summary()
  attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  filename = output_attack_dir + "/" + recipe + "eval_whole" 

  create_csv_file(attacked_dict,filename+"_attacked.csv")



In [69]:
#multi_eval_dataset = pd.DataFrame.from_dict(multi_eval_dataset)
#snli_eval_dataset = pd.DataFrame.from_dict(snli_eval_dataset)
#classifier = '../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing'
#attack_recipe("random_nli",snli_eval_dataset,"snli",classifier,model_from_file=None, model=None,model_from_huggingface=classifier,model_batch_size=32)


Output hidden; open in https://colab.research.google.com to view.

In [76]:
!ls ../TrainedBertModels/outputs/training/bertbase_max_length64/bert-base-uncased-snli-lr3e5_batchsize32_maxlength64

1597448743.868051				   README.md
config.json					   special_tokens_map.json
events.out.tfevents.1597448743.a5d2f71e29f2.534.0  tokenizer_config.json
log.txt						   train_args.json
pytorch_model.bin				   vocab.txt


In [79]:
!ls ../TrainedBertModels/outputs/training/

bertbase_maxlength128  bert-base-uncased-snli-testing
bertbase_max_length64  roberta-base-snli-2020-08-22-13-27-50-640954


## Get prectiles for delta tuning

In [ ]:
snli_dataset = nlp.load_dataset("snli")
multi_dataset = nlp.load_dataset("multi_nli")
multi_train_dataset = return_multi_nli_train_dict("multi_nli")
multi_eval_dataset = return_multi_nli_eval_dict("multi_nli")
percentiles = [0,10,20,30,40,50,60,70,80,90,100]
# split='validation_matched', ,split='validation_mismatched')

----EVAL DATASET USING : 58906

----EVAL DATASET USING : 58906



In [ ]:
import nltk

def _get_modifiable_indices(words):
  """Returns the word indices in ``current_text`` which are able to be
  modified."""
  stopwords = set(nltk.corpus.stopwords.words("english"))

  non_stopword_indices = set()
  for i, word in enumerate(words):
      if word not in stopwords:
          non_stopword_indices.add(i)
  return non_stopword_indices


In [ ]:
# SNLI 
frequencies_train = []
for sentence in snli_dataset['train']['hypothesis']:
  words = words_from_text(sentence)
  phi_values = get_phi_values(words,snli_freq_dict)
  frequencies_train += phi_values

print("SNLI frequency precentiles")
snli_precentiles = np.percentile(frequencies_train,percentiles)
print(snli_precentiles)

SNLI frequency precentiles
[ 0.69314718  6.3526294   7.83636976  8.92532142  9.80477158 10.72827684
 11.64400636 12.20202446 12.82231448 13.15694643 13.73591308]


In [ ]:
# MNLI 
frequencies_train = []
for sentence in multi_train_dataset['hypothesis']:
  words = words_from_text(sentence)
  phi_values = get_phi_values(words,multi_freq_dict)
  frequencies_train += phi_values

print("MNLI frequency precentiles")
mnli_precentiles = np.percentile(frequencies_train,percentiles)
print(mnli_precentiles)

MNLI frequency precentiles
[ 0.69314718  4.73619845  6.12905021  7.01301579  7.81883244  8.79270146
  9.81536647 10.5948832  11.3667661  12.44278374 13.15266365]


In [ ]:
len(multi_train_dataset['hypothesis'])

338063

In [ ]:
len(multi_eval_dataset['hypothesis'])

58906

## Number of candidates for WordSwapEmbedding for frequency attacks

In [ ]:
from textattack.transformations import WordSwapWordNet
wordnet = WordSwapWordNet()

In [ ]:

num_candidates = []
for words in snli_dataset['train']['hypothesis']:
  tokens = words_from_text(words)
  num_candidates += [len(wordnet._get_replacement_words(word)) for word in tokens]

print(np.mean(np.array(num_candidates)))

8.042087898972378


In [ ]:
num_candidates = []
for words in snli_dataset['validation']['hypothesis']:
  tokens = words_from_text(words)
  num_candidates += [len(wordnet._get_replacement_words(word)) for word in tokens]

print(np.mean(np.array(num_candidates)))

8.04590451860131


In [ ]:

num_candidates = []
for words in snli_dataset['test']['hypothesis']:
  tokens = words_from_text(words)
  num_candidates += [len(wordnet._get_replacement_words(word)) for word in tokens]

print(np.mean(np.array(num_candidates)))


8.021868999359931


In [ ]:

num_candidates = []
for words in multi_dataset['train']['hypothesis']:
  tokens = words_from_text(words)
  num_candidates += [len(wordnet._get_replacement_words(word)) for word in tokens]

print(np.mean(np.array(num_candidates)))


6.891560975858621


In [ ]:

num_candidates = []
for words in multi_eval_dataset['hypothesis']:
  tokens = words_from_text(words)
  num_candidates += [len(wordnet._get_replacement_words(word)) for word in tokens]

print(np.mean(np.array(num_candidates)))

6.89493842427117


In [ ]:

num_candidates = []
for words in multi_train_dataset['hypothesis']:
  tokens = words_from_text(words)
  num_candidates += [len(wordnet._get_replacement_words(word)) for word in tokens]

print(np.mean(np.array(num_candidates)))


6.891596554439664


In [ ]:

num_candidates = []
for words in multi_dataset['validation_matched']['hypothesis']:
  tokens = words_from_text(words)
  num_candidates += [len(wordnet._get_replacement_words(word)) for word in tokens]

print(np.mean(np.array(num_candidates)))


6.9044042195131965


In [ ]:

num_candidates = []
for words in multi_dataset['validation_mismatched']['hypothesis']:
  tokens = words_from_text(words)
  num_candidates += [len(wordnet._get_replacement_words(word)) for word in tokens]

print(np.mean(np.array(num_candidates)))


6.9645748229209605


# NWS

In [ ]:
EvalDefenseModel(defense_from_class=["../detect_adversarial", "NWS"],
      model_from_file=None, 
      model=None,
      model_from_huggingface='../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing',
      model_batch_size=32,
      delta = phi_value(50),
      type = 'continuous',
      gamma = 0.1,
      freq_dict = snli_freq_dict,
      data_file='../TrainedBertModels/outputs/training/bert-base-uncased-snli-testing/attack/alzantot_gpt2_2000_attacked.csv',
      data_name = 'snli',
      candidates = 8)

Output hidden; open in https://colab.research.google.com to view.